# <span style="font-width:bold; font-size: 3rem; color:#1EB182;">**Hopsworks Feature Store** </span><span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Feature Pipeline</span>


## 🗒️ This notebook is divided into 2 sections:
1. Data generation.
2. Insert new data into the Feature Store.

### <span style='color:#ff5f27'> 📝 Imports

In [ ]:
import pandas as pd
from datetime import datetime
import time 
import os 

from features import (
    nyc_taxi_rides, 
    nyc_taxi_fares,
)

# Mute warnings
import warnings
warnings.filterwarnings("ignore")

___

## <span style="color:#ff5f27;"> 🪄 Generating new data</span>

### <span style='color:#ff5f27'> 🚖 Rides Data

In [ ]:
# Generate ride data using the generate_rides_data function
df_rides = nyc_taxi_rides.generate_rides_data(150)

# Display the DataFrame containing the generated ride data
df_rides.head(5)

In [ ]:
# Calculate distance-related features for the df_rides DataFrame
df_rides = nyc_taxi_rides.calculate_distance_features(df_rides)

# Calculate datetime-related features for the df_rides DataFrame
df_rides = nyc_taxi_rides.calculate_datetime_features(df_rides)

In [ ]:
# Save the newly-generated ride_ids from the df_rides DataFrame
ride_ids = df_rides.ride_id

In [ ]:
# Convert specified columns in df_rides DataFrame to int64 type
for col in ["passenger_count", "taxi_id", "driver_id"]:
    df_rides[col] = df_rides[col].astype("int64")

### <span style='color:#ff5f27'> 💸 Fares Data

In [ ]:
# Generate fare data using the generate_fares_data function
df_fares = nyc_taxi_fares.generate_fares_data(150)

# Display the DataFrame containing the generated fare data
df_fares.head()

In [ ]:
# Convert all columns in the df_fares DataFrame to int64 type
df_fares = df_fares.astype("int64")

In [ ]:
# Load our ride_ids which were created moments ago for rides_fg
df_fares["ride_id"] = ride_ids

In [ ]:
# Convert specified columns in df_fares DataFrame to float64 type
for col in ["tolls", "total_fare"]:
    df_fares[col] = df_fares[col].astype("float64")

___

## <span style="color:#ff5f27;"> 📡 Connecting to the Hopsworks Feature Store </span>

In [ ]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store() 

In [ ]:
rides_fg = fs.get_feature_group(
    name="nyc_taxi_rides",
    version=1,
)   

fares_fg = fs.get_feature_group(
    name="nyc_taxi_fares",
    version=1,
)   

---

## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [ ]:
rides_fg.insert(df_rides)

In [ ]:
fares_fg.insert(df_fares)

---

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline </span>

In the next notebook, you will create a feature view, training dataset, train a model and save it to Hopsworks Model Registry.
